In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)

import os
import glob

In [2]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

## Functions for preprocess

In [3]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [4]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [5]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [6]:
def count_unique(series):
    return len(np.unique(series))

## Main function for preprocessing book data

In [7]:
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    #calculate return etc
    df['wap'] = calc_wap(df)
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    #dict for aggregate
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap':[np.mean],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

In [8]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path)

CPU times: user 5.03 s, sys: 71.3 ms, total: 5.1 s
Wall time: 5.04 s


,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,log_return_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,row_id
0,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0.002953,0.004864,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0-5
1,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0-11
2,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0-16
3,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0-31
4,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002579,0.003821,0.000212,0.000552,0.000083,-0.000182,197.144781,374.235690,0.997938,0.001673,0.002573,0.000193,0.000509,0.000062,-0.000169,233.946667,350.560000,0.997519,0-32751
3826,0.002206,0.002847,0.000267,0.000542,0.000092,-0.000172,233.781553,621.131068,1.000310,0.001487,0.002255,0.000300,0.000588,0.000074,-0.000177,257.920000,668.640000,1.000682,0-32753
3827,0.002913,0.003266,0.000237,0.000525,0.000202,-0.000083,115.829787,343.734043,0.999552,0.001928,0.002646,0.000216,0.000446,0.000191,-0.000075,105.432692,326.759615,1.000111,0-32758
3828,0.003046,0.005105,0.000245,0.000480,0.000113,-0.000166,132.074919,385.429967,1.002357,0.002137,0.003934,0.000269,0.000516,0.000096,-0.000175,123.423313,394.588957,1.002277,0-32763


## Main function for preprocessing trade data

In [9]:
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [10]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

CPU times: user 2.57 s, sys: 9.13 ms, total: 2.58 s
Wall time: 2.56 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,row_id
0,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429,0-5
1,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000,0-11
2,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667,0-16
3,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111,0-31
4,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091,0-62
...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.001162,35.0,2365.0,3.257143,0-32751
3826,0.001411,28,4547,3.892857,0.001066,12.0,2161.0,4.250000,0-32753
3827,0.001521,36,4250,3.500000,0.001242,22.0,2294.0,3.727273,0-32758
3828,0.001794,53,3217,2.150943,0.001404,25.0,1627.0,1.920000,0-32763


## Combined preprocessor function

In [11]:
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [12]:
list_stock_ids = [0,1]
preprocessor(list_stock_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    8.1s finished


,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,log_return_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,row_id,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0.002953,0.004864,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0-5,0.002006,40,3179,2.750000,0.001308,21.0,1587.0,2.571429
1,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0-11,0.000901,30,1289,1.900000,0.000587,16.0,900.0,2.250000
2,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0-16,0.001961,25,2161,2.720000,0.001137,12.0,1189.0,3.166667
3,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0-31,0.001561,15,1962,3.933333,0.001089,9.0,1556.0,5.111111
4,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0-62,0.000871,22,1791,4.045455,0.000453,11.0,1219.0,4.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,0.003723,0.004996,0.000330,0.000597,0.000157,-0.000118,125.013029,296.185668,1.000142,0.002212,0.002954,0.000283,0.000617,0.000165,-0.000104,120.469231,322.284615,1.000130,1-32751,0.001776,49,3249,2.775510,0.001280,23.0,1889.0,3.608696
7656,0.010829,0.012168,0.000403,0.000922,0.000159,-0.000125,254.006073,567.840081,1.007503,0.008499,0.009971,0.000483,0.001082,0.000196,-0.000129,217.410788,485.195021,1.012343,1-32753,0.008492,183,75903,7.874317,0.006310,88.0,30858.0,8.136364
7657,0.003135,0.004268,0.000243,0.000648,0.000141,-0.000132,163.645367,426.603834,1.000854,0.002108,0.003184,0.000261,0.000625,0.000146,-0.000146,142.402685,426.939597,1.001250,1-32758,0.001927,26,2239,2.615385,0.001567,11.0,980.0,2.727273
7658,0.003750,0.005773,0.000199,0.000421,0.000190,-0.000231,138.235023,526.317972,1.003032,0.002728,0.004435,0.000195,0.000410,0.000165,-0.000240,148.273543,519.044843,1.004296,1-32763,0.002856,109,16648,2.935780,0.001919,57.0,8274.0,2.701754


## Preprocess Training set

In [13]:
train = pd.read_csv(data_dir + 'train.csv')

In [14]:
train_ids = train.stock_id.unique()

In [15]:
%%time
df_train = preprocessor(list_stock_ids= train_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   41.4s


CPU times: user 1.05 s, sys: 46.9 ms, total: 1.1 s
Wall time: 2min


[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  2.0min finished


In [16]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_train = train.merge(df_train, on = ['row_id'], how = 'left')

In [17]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,log_return_realized_volatility_300,log_return2_realized_volatility_300,wap_balance_mean_300,price_spread_mean_300,bid_spread_mean_300,ask_spread_mean_300,volume_imbalance_mean_300,total_volume_mean_300,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0-5,0.004136,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0.002953,0.004864,0.000372,0.000822,0.000223,-0.000162,137.158273,294.928058,1.003753,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429
1,0-11,0.001445,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0.000981,0.002009,0.000239,0.000353,0.000164,-0.000123,135.513043,484.521739,1.000397,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000
2,0-16,0.002168,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0.001295,0.003196,0.000431,0.000689,0.000141,-0.000249,144.147059,455.235294,0.998685,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667
3,0-31,0.002195,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0.001776,0.002713,0.000331,0.000833,0.000158,-0.000095,144.698113,418.169811,0.998436,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111
4,0-62,0.001747,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0.001520,0.002188,0.000252,0.000425,0.000191,-0.000120,99.449438,407.584270,0.999488,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091


## Preprocess Test set

In [18]:
test = pd.read_csv(data_dir + 'test.csv')

In [19]:
test_ids = test.stock_id.unique()

In [20]:
%%time
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)

CPU times: user 10.4 ms, sys: 182 µs, total: 10.6 ms
Wall time: 80.2 ms


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [21]:
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

In [22]:
from sklearn.model_selection import KFold
#stock_id target encoding
df_train['stock_id'] = df_train['row_id'].apply(lambda x:int(x.split('-')[0]))
df_test['stock_id'] = df_test['row_id'].apply(lambda x:int(x.split('-')[0]))

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean() 
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean) # test_set

#training
tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits = 10, shuffle=True,random_state = 19911109)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()

    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

## Build NN

In [23]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import backend as K
from sklearn import preprocessing, model_selection

In [24]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [25]:
hidden_units = (32,16,8,4,2)
stock_embedding_size = 16

cat_data = df_train['stock_id']

def base_model():
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(3,), name='num_data')


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='selu')(out)
        

    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [26]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=1e-05, patience=10, verbose=1,
    mode='min', baseline=0.25)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, verbose=1,
    mode='min')

In [27]:
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)
target_name = 'target'

n_folds = 4
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=2020)
scores_folds = {}
scores_folds[model_name] = []
counter = 1

features_to_consider = ['stock_id','log_return_realized_volatility','log_return2_realized_volatility','trade_log_return_realized_volatility']

df_train[pred_name] = 0
df_test['target'] = 0

df_train['trade_log_return_realized_volatility'] = df_train['trade_log_return_realized_volatility'].fillna(0)
df_test['log_return_realized_volatility'] = df_test['log_return_realized_volatility'].fillna(0)
df_test['log_return2_realized_volatility'] = df_test['log_return2_realized_volatility'].fillna(0)
df_test['trade_log_return_realized_volatility'] = df_test['trade_log_return_realized_volatility'].fillna(0)


for dev_index, val_index in kf.split(range(len(train))):
    print('CV {}/{}'.format(counter, n_folds))
    
    #Bottleneck ? 
    X_train = df_train.loc[dev_index, features_to_consider]
    y_train = df_train.loc[dev_index, target_name].values
    X_test = df_train.loc[val_index, features_to_consider]
    y_test = df_train.loc[val_index, target_name].values
    
    #############################################################################################
    # NN
    #############################################################################################
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.01),
        loss=tf.keras.metrics.mean_squared_error,
        metrics=['MSE'],
    )


    num_data = X_train[['log_return_realized_volatility','log_return2_realized_volatility','trade_log_return_realized_volatility']]
    cat_data = X_train['stock_id']
    target =  y_train
    
    num_data_test = X_test[['log_return_realized_volatility','log_return2_realized_volatility','trade_log_return_realized_volatility']]
    cat_data_test = X_test['stock_id']

    model.fit([cat_data, num_data], 
              target, 
              sample_weight = 1/np.square(target),
              batch_size=1024,
              epochs=100,
              validation_data=([cat_data_test, num_data_test], y_test, 1/np.square(y_test)),
              callbacks=[es, plateau],
              shuffle=True,
             verbose = 1)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    df_test[target_name] += model.predict([df_test['stock_id'], df_test[['log_return_realized_volatility','log_return2_realized_volatility','trade_log_return_realized_volatility']]]).reshape(1,-1)[0].clip(0,1e10)
       
    counter += 1

CV 1/4
Epoch 1/100
315/315 [==============================] - 10s 29ms/step - loss: 1002.8080 - MSE: 0.0047 - val_loss: 0.1603 - val_MSE: 6.9978e-06
Epoch 2/100
315/315 [==============================] - 1s 3ms/step - loss: 0.1528 - MSE: 6.8439e-06 - val_loss: 0.1454 - val_MSE: 5.5185e-06
Epoch 3/100
315/315 [==============================] - 1s 2ms/step - loss: 0.1327 - MSE: 5.8025e-06 - val_loss: 0.1233 - val_MSE: 5.4021e-06
Epoch 4/100
315/315 [==============================] - 1s 2ms/step - loss: 0.1120 - MSE: 4.7222e-06 - val_loss: 0.1030 - val_MSE: 3.6047e-06
Epoch 5/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0953 - MSE: 3.7639e-06 - val_loss: 0.1528 - val_MSE: 2.6250e-06
Epoch 6/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0938 - MSE: 3.1178e-06 - val_loss: 0.1211 - val_MSE: 2.3182e-06
Epoch 7/100
315/315 [==============================] - 1s 3ms/step - loss: 0.0964 - MSE: 2.7229e-06 - val_loss: 0.1098 - val_MSE: 3.0784e-06

Epoc

315/315 [==============================] - 1s 2ms/step - loss: 0.1069 - MSE: 2.6319e-06 - val_loss: 0.1762 - val_MSE: 2.2882e-06
Epoch 7/100
315/315 [==============================] - 1s 2ms/step - loss: 0.1926 - MSE: 2.9974e-06 - val_loss: 0.0745 - val_MSE: 2.5116e-06
Epoch 8/100
315/315 [==============================] - 1s 2ms/step - loss: 0.3117 - MSE: 3.5957e-06 - val_loss: 0.3393 - val_MSE: 2.5213e-06
Epoch 9/100
315/315 [==============================] - 1s 2ms/step - loss: 0.3309 - MSE: 3.7013e-06 - val_loss: 0.4637 - val_MSE: 2.9674e-06
Epoch 10/100
315/315 [==============================] - 1s 2ms/step - loss: 0.3045 - MSE: 3.5813e-06 - val_loss: 0.1677 - val_MSE: 2.4748e-06

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
Epoch 11/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0806 - MSE: 2.7609e-06 - val_loss: 0.0718 - val_MSE: 2.5304e-06
Epoch 12/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0708 

Epoch 14/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0694 - MSE: 2.6706e-06 - val_loss: 0.0669 - val_MSE: 2.5129e-06
Epoch 15/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0716 - MSE: 2.6392e-06 - val_loss: 0.1055 - val_MSE: 2.3506e-06
Epoch 16/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0739 - MSE: 2.5548e-06 - val_loss: 0.0659 - val_MSE: 2.5015e-06
Epoch 17/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0714 - MSE: 2.5351e-06 - val_loss: 0.0875 - val_MSE: 2.3082e-06
Epoch 18/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0783 - MSE: 2.5665e-06 - val_loss: 0.0655 - val_MSE: 2.4685e-06
Epoch 19/100
315/315 [==============================] - 0s 2ms/step - loss: 0.0704 - MSE: 2.5432e-06 - val_loss: 0.1290 - val_MSE: 3.3004e-06
Epoch 20/100
315/315 [==============================] - 0s 1ms/step - loss: 0.0772 - MSE: 2.5186e-06 - val_loss: 0.0719 - val_MSE: 2.3775e-06
Epoch 

315/315 [==============================] - 1s 3ms/step - loss: 0.1267 - MSE: 3.6120e-06 - val_loss: 0.1665 - val_MSE: 3.8099e-06
Epoch 17/100
315/315 [==============================] - 1s 3ms/step - loss: 0.1200 - MSE: 3.4996e-06 - val_loss: 0.1181 - val_MSE: 4.0099e-06
Epoch 18/100
315/315 [==============================] - 1s 3ms/step - loss: 0.1200 - MSE: 3.4391e-06 - val_loss: 0.1106 - val_MSE: 3.6595e-06

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
Epoch 19/100
315/315 [==============================] - 1s 3ms/step - loss: 0.0883 - MSE: 3.0883e-06 - val_loss: 0.1004 - val_MSE: 3.0800e-06
Epoch 20/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0914 - MSE: 3.0745e-06 - val_loss: 0.0863 - val_MSE: 3.0553e-06
Epoch 21/100
315/315 [==============================] - 1s 2ms/step - loss: 0.0880 - MSE: 3.0741e-06 - val_loss: 0.0857 - val_MSE: 2.8983e-06
Epoch 22/100
315/315 [==============================] - 1s 4ms/step - loss: 0.09

## Predict Test set

In [28]:
df_test.columns

Index(['stock_id', 'time_id', 'row_id', 'log_return_realized_volatility',
       'log_return2_realized_volatility', 'wap_balance_mean',
       'price_spread_mean', 'bid_spread_mean', 'ask_spread_mean',
       'volume_imbalance_mean', 'total_volume_mean', 'wap_mean',
       'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'trade_log_return_realized_volatility',
       'trade_seconds_in_bucket_count_unique', 'trade_size_sum',
       'trade_order_count_mean', 'trade_log_return_realized_volatility_300',
       'trade_seconds_in_bucket_count_unique_300', 'trade_size_sum_300',
       'trade_order_count_mean_300', 'stock_id_target_enc', 'target'],
      dtype='object')

In [29]:
df_train.columns

Index(['row_id', 'target', 'log_return_realized_volatility',
       'log_return2_realized_volatility', 'wap_balance_mean',
       'price_spread_mean', 'bid_spread_mean', 'ask_spread_mean',
       'volume_imbalance_mean', 'total_volume_mean', 'wap_mean',
       'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'trade_log_return_realized_volatility',
       'trade_seconds_in_bucket_count_unique', 'trade_size_sum',
       'trade_order_count_mean', 'trade_log_return_realized_volatility_300',
       'trade_seconds_in_bucket_count_unique_300', 'trade_size_sum_300',
       'trade_order_count_mean_300', 'stock_id', 'stock_id_target_enc',
       'pred_NN'],
      dtype='object')

In [30]:
from IPython.core.display import display


df_test[target_name] = df_test[target_name]/n_folds

score = round(rmspe(y_true = df_train[target_name].values, y_pred = df_train[pred_name].values),5)
print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

display(df_test[['row_id', target_name]].head(2))
df_test[['row_id', target_name]].to_csv('submission.csv',index = False)

RMSPE NN: 1.0 - Folds: [0.25662, 0.25841, 0.25255, 0.28152]


,row_id,target
0,0-4,0.000711
1,0-32,0.000520
